# Graph Neural Networks


---
**Lernziele**
* Sie verstehen, wie Moleküle als Graphen für Computer dargestellt werden können. 
* Sie verstehen, wie Graph Neural Networks funktionieren.
* Sie sind in der Lage, ein Graph Neural Network als Pytorch Klasse zu schreiben.
---


Graph Neural Networks sind noch eine relativ neue Methode. Intuitiv lassem sich Moleküle sehr gut als (mathematischer) Graph dargestellt. Die Bindungen eines Molekül entsprechen den Kanten des Graphen und die Atome den Knoten. <br> Für den Computer sind Graphen jedoch nicht so einfach zu lesen wie z. B. ein Smiles `string`. Für Graph Neurale Netzwerke stellen die Moleküle durch mindestens zwei Matrizen dar. Die eine ist die Adjacencymatrix, die die Verbindungen zwischen den Atome (d. h. die Bindungen) darstellt. Die andere Matrix ist die Featurematrix. Hier können Informationen zu einzelnen Atomen gespeichert werden.


<center>
<img src="https://www.researchgate.net/profile/Jorge_Galvez2/publication/236018587/figure/fig1/AS:299800013623305@1448489301609/The-chemical-graph-and-adjacency-matrix-of-the-isopentane.png" style="width: 600px;">
</center>
<h8><center>Galvez et. al. 2010</center></h8><br><br>

Für das heutige Beispiel benutzen wie nochmal die Daten aus der Tox21 Challenge. 

In [1]:
import pandas as pd
import numpy as np
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils import data
import math
from sklearn.metrics import roc_auc_score
from os.path import exists
import sys
if 'google.colab' in sys.modules:
    !pip install rdkit==2022.3.4
    if exists("utils.py") == False:
        !wget https://raw.githubusercontent.com/kochgroup/intro_pharma_ai/main/utils/onehotencoder.py
    %run ../utils/onehotencoder.py

else:
    %run ../utils/onehotencoder.py
from rdkit import Chem
from rdkit.Chem.rdmolops import GetAdjacencyMatrix

np.set_printoptions(linewidth=300)

In [2]:
# Laden der Daten
data_tox = pd.read_csv("https://raw.githubusercontent.com/filipsPL/tox21_dataset/master/compounds/sr-mmp.tab", sep = "\t")
data_tox = data_tox.iloc[:,1:] #alle Spalten bis auf die erste (index 0) werden ausgewählt
data_tox.columns = ["smiles", "activity"]
data_tox.head()

,smiles,activity
0,OC(=O)[C@H](O)[C@@H](O)[C@H](O)C(=O)CO,0
1,C[C@]12CC[C@H]3[C@@H](CCc4cc(O)ccc43)[C@@H]1CC...,1
2,CC(C)(C)c1cc(O)ccc1O,1
3,CN(C)c1ccc(cc1)C(c1ccccc1)=C1C=CC(C=C1)=[N+](C)C,1
4,NC(Cc1ccccc1)C(O)=O,0


## Adjacency Matrix und One-Hot Encoded Feature Matrix

Leider können wir dieses Mal nicht viel mit den Smiles anfangen. Dafür gibt es in RDKit Funktionen, die es uns erleichtern mit Graphen zu arbeiten. Zum Beispiel gibt es eien Funktion, die Adjacencymatrix für Moleküle erstellt. Wir haben diese bereits oben importiert.

In [16]:
mols = [Chem.MolFromSmiles(x) for x in data_tox['smiles']]
A = [GetAdjacencyMatrix(x) for x in mols]
print(A[1])

[[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 


Als Features für die Atome benutzen wir nur den Atomtype. Diese werdem wir auch One-Hot kodieren.

Für die One-Hot Kodierung der Atome verwenden wir die bereits geschriebene Funktion `onehotencode()`.
Ähnlich wie bei der Kodierung von Smilestokens in RNNs wird hier nur der Atomtyp eines jeden Atoms erfasst. Diese Atome werden dann als One-Hot kodierte Vektoren dargestellt.
Pro Molekül werden diese Vektoren miteinander kombiniert, um eine Matrix zu bilden.

In [17]:
feat = onehotencode(mols)
feat[1]

array([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Oben können Sie sehen, wie eine Featurematrix für ein Molekül aussieht.
Wenn wir uns die `.shape` ansehen, können wir sehen, dass dieses Molekül aus `29` Atomen besteht. Die Anzahl der Spalten lässt uns wissen, dass es insgesamt 25 Atomtypen im Datensetz gibt. Die Anzahl der Spalten (Anzahl der Features) muss für alle Moleküle gleich sein, sonst können wir die Moleküle nicht durch das Netzwerk führen.

In [18]:
feat[1].shape

(29, 25)

Sie haben vielleicht bemerkt, dass auf der Diagonalen der Adjacencymatrix noch Nullen stehen. Bei einer Graph Convolution sollen aber nicht nur die Features der Nachbaratome, sondern auch die des Zentralatoms in die Berechnung einbezogen werden. Hierfür werden Einsen auf der Diagonalen der Adjacencymatrix benötigt. 
Mit der Funktion `np.fill_diagonal(matrix, value)` können Sie die Werte der Diagonalen einer Matrix ändern. 

In [19]:
for matrix in A:
    np.fill_diagonal(matrix, 1)
print(A[1])

[[1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 

## Graph Convolution

Nun wollen wir die Informationen der Knoten an die Nachbarknoten entlang der Kanten weitergeben. Das wird (Graph-)Convolution bezeichnet. Dazu werden bei der Forward Propagation diese mathematischen Operationen durchgeführt: $$\hat{X} = \hat{D}^{-1}\hat{A}XW$$.

Dabei ist $\hat{A}$ unsere Adjazcenymatrix, mit Einsen auf der Diagonalen. $X$ ist die Featurematrix und $W$ sind die Weights, die Pytorch initialisiert. Was noch fehlt, ist $D$, die $D$egreematrix. Diese Matrix enthält die Anzahl der Bindungen, die jedes Atom im Molekül hat. Diese Werte werden auf der Diagonalen der Degreematrix platziert.
Die Anzahl der Verbindungen eines jeden Moleküls lässt sich leicht aus der Adjacency Matrix berechnen. Dazu muss man die Summe über die  Zeilen oder Spalten der Matrix $\hat{A}$ berechnen. Diese Summe ergibt den Degree eines Atoms. Um genau zu sein, ist es der Degree plus eins, da man die Diagonale der Adjacencymatrix bereits mit Einsen gefüllt hat.
Um die Degree-Matrix zu erstellen, müssen Sie die Summen der Spalten in jeder Adjacenymatrix berechnen und diese auf die Diagonale einer neuen Matrix setzen.


Dieser Vorgang ist mit `numpy` relativ einfach durchzuführen. `np.sum(A[i], axis=0)` berechnet die Summe pro Spalte und `np.diag()` erzeugt eine Matrix aus einem 1D-Array mit den Werten des 1D-Arrays auf der Diagonalen. Sie können die beiden Funktionen in Kombination verwenden, um die Degreematrizen zu erstellen.

In [20]:
D =[]
for matrix in A:
    D.append(np.diag(np.sum(matrix, axis=1)))
print(D[1])

[[2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 5 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 


Aber wir brauchen $\hat{D}$ nicht. Wir brauchen das Inverse dieser Matrix. Ohne $\hat{D}^{-1}$ würde $\hat{A}X$ die Features über alle verbundenen Knoten summieren. Dies würde dazu führen, dass Atome mit mehr Nachbarn größere Featurewerte haben. Durch die Einbeziehung von $\hat{D}^{-1}$ werden die aggregierten Merkmale durch die Anzahl der benachbarten Atome gemittelt. `np.linalg.inv()` wird benötigt, um die Inverse der Matrix `D`.

Bevor wir mit dem Aufbau eines Netzes beginnen, müssen wir einen weiteren Schritt durchführen. Um Rechenaufwand zu sparen, können wir die Berechnung $\hat{D}^{-1}\hat{A}$ bereits vor dem Training des Netzes berechnen. Dies kann Zeit sparen, da dieser Schritt nicht immer wieder im Netz wiederholt werden muss.





In [21]:
DA = []
for i in range(len(D)):
    DA.append(np.matmul(np.linalg.inv(D[i]),A[i]))
DA[0]

array([[0.5       , 0.5       , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.25      , 0.25      , 0.25      , 0.25      , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.5       , 0.5       , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.25      , 0.        , 0.25      , 0.25      , 0.25      , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.5       , 0.5       , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.25      , 0.        , 0.25      , 0.25      , 0.25      , 0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.

Wir haben also die Liste der Adjacencymatrizen `DA`. Diese enthält die Informationen über die Struktur des Moleküls. Die Liste `feat` enthält die Features, d.h. die Informationen über die einzelnen Atome. Und schließlich erstellen wir eine Liste `labels`. Diese enthält die Informationen, die wir vorhersagen wollen (toxisch vs. nicht toxisch).

In [22]:
DA = [torch.tensor(x,dtype=torch.float32) for x in DA] # Konvertieren der Arrays zu Tensoren
feat = [torch.tensor(x,dtype=torch.float32) for x in feat] # Konvertieren der Arrays zu Tensoren
labels = [torch.tensor([x], dtype=torch.float32) for x in data_tox['activity']]

## Graph Convolution Layer

Wir wollen uns die Struktur von PyTorch für die Graph Convolution zunutze machen. Wie letzte Woche werden wir dafür Klassen verwenden.
Letzte Woche haben wir eine Klasse benutzt, um ein Netzwerk aus verschiedenen Layern zu erstellen (z.B. `nn.Linear()`). 
Diese Layers sind in PyTorch bereits vorgegeben, aber wir können auch unsere eigenen Layers erstellen. Auch hierfür benötigen wir die PyTorch-Klasse.
Unser Ziel ist es, so etwas wie `nn.Linear()` zu programmieren, damit wir unser Repertoire an Layers (Linear, RNN, GRU, Dropout, ...) um eine Graph Convolution erweitern können.

Auch hierfür können wir die Klasse `nn.Module` als Basis für unsere Graph Convolution verwenden. Wir beginnen mit dem Minimalgerüst:


```python
class GraphConvolution(nn.Module):
    pass
```

Damit wir richtig auf die Funktionen der übergeordneten `nn.Module` Klasse zugreifen können, müssen wir diese mit `super().__init__()` initialisieren. Unsere Convolutional Layer muss natürlich wissen, wie groß der Input ist, und wie groß der Output werden soll. Da wir jetzt nicht ein Netzwerk, sondern nur eine Layer definieren, müssen wir zusätzlich angeben wie die Weights und Biases zu initialisieren sind.

```python
    def __init__(self, in_features, out_features):
        super().__init__()
        self.in_featuers = in_features
        self.out_features = out_features
        self.weight = nn.Parameter(torch.FloatTensor(in_features, out_features))
        self.bias = nn.Parameter(torch.FloatTensor(out_features))
```

Dann können wir auch schon die `forward()` Funktion programmieren. Hier passiert die eigentliche Convolution, bei der die Knoten mit den Features `x` durch Matrixmultiplikation mit der Adjacency Matrix `adj` über ihre Nachbarknoten aggregiert werden. Indem wir noch die lernbaren `weights` dazwischenschalten, können wir das Ganze später optimieren.

```python
    def forward(self, x, adj):
        support = torch.mm(x, self.weight)
        output = torch.mm(adj, support)
        return output + self.bias
```
Damit würde unsere `GraphConvolution` Klasse auch schon funktionieren. Wir initialisieren die Weights mit der Funktion `reset_parameters()`. Die Funktion `__repr__()` gibt graphisch wieder wie unsere Layer aussieht, nachdem sie initilaiert wurde. 

Unsere finale Klasse sieht dann so aus:

In [23]:
class GraphConvolution(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = nn.Parameter(torch.FloatTensor(in_features, out_features))
        self.bias = nn.Parameter(torch.FloatTensor(out_features))
        self.reset_parameters()
    
    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(0, stdv)
        self.bias.data.uniform_(-stdv, stdv)
        
    def forward(self, x, adj):
        support = torch.mm(x, self.weight)
        output = torch.mm(adj, support)
        return output + self.bias
    
    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + 'in_features=' + str(self.in_features) + ', ' \
               + 'out_features=' + str(self.out_features) + ')'

Wir können auch überprüfen, ob unsere Graph Convolution Layer auch schon funktioniert.
Zunächst speichern wir eine Feature-Matrix und eine Adjacency Matrix als Beispiel.

In [24]:
feat_beispiel = feat[1]
adj_beispiel = DA[1]
print('Features:', feat_beispiel.shape)
print('DA:', adj_beispiel.shape)

Features: torch.Size([29, 25])
DA: torch.Size([29, 29])


Wir erstellen nun eine `GraphConvolution`. Beachten Sie, dass die Inputgröße, der ersten Layer der Größe des Featurevektors (Anzahl der Spalten) (`25`) entspricht. Also genauso wie bei einer `nn.Linear` Layer.

In [25]:
conv = GraphConvolution(25, 100)
conv

GraphConvolution (in_features=25, out_features=100)

Wir können nun das Beispiel durch die Convolution führen. Sie werden sehen, dass sich die Dimension der Features auf 100 vergrößert haben wird.

In [ ]:
output = conv(feat_beispiel, adj_beispiel)
print('\nOutput:', output.size())

## Graph Neural Network

Um nun ein Netz zu erstellen, können wir wieder die Klasse `nn.Module` verwenden. 

Letzte Woche haben wir unseren eigenen Autoencoder zusammengestellt. Diese Woche verwenden wir diese Art von Klasse, um mehrere Graph Convolutions miteinander zu verbinden. Es ist jedoch wichtig zu beachten, dass wir, wie bei regulären CNN, nicht nur Convolution Layers verwenden können. Im CNN selbst müssen wir den Tensor am Ende "flatten". Dadurch erhalten wir einen Vektor, der dann durch eine lineare Layer geleitet wird. Das Gleiche gilt für Graph Convolutions. Um die Ausgabe dieser Graph Convolutions in eine lineare Layer zu leiten, können wir jedoch nicht einfach PyTorchs `flatten` verwenden, sondern wir werden den Durchschnitt Durchschnitt über alle Spalten berrechnen.

Dies geschieht mit der Funktion `aggregate()`.

Sie können im Netzwerk sehen, dass wir zwei Graph Convolution und dann eine lineare Layer verwenden.
Im Forward Pass wird zunächst der Input `(x, adj)` durch die erste Graph Convolution geführt, dann folgt eine ReLU-Funktion. Das gleiche Spiel wird dann für die zweite Convolution wiederholt. 
Nun wenden wir die Funktion `aggregate` an. Diese errechnet nun den Mittelwert jedes Features. Der Output dieser Layer hat also im Beispiel die Größe `[1,100]`.
Schließlich verwenden wir die lineare Layer, um die eigentliche Vorhersage zu treffen.

In [28]:
class GraphNN(nn.Module):
    def __init__(self):#in_features, out_features, size_labels):
        super().__init__()
        self.conv1 = GraphConvolution(25, 100)
        self.conv2 = GraphConvolution(100, 100)
        self.lin = nn.Linear(100, 1)
        
    def aggregate(self, convoluted_graph): # we use mean aggregation, max or min could also be used as hyperparameter
        return torch.mean(convoluted_graph, dim=0, keepdim=True)
        
    def forward(self, x, adj):
        x = self.conv1(x, adj)
        x = F.relu(x)
        x = self.conv2(x, adj)
        x = F.relu(x)
        x = self.aggregate(x)
        x = self.lin(x)
        return x

Nun werden wir den Datensatz schnell in einen Trainings- und einen Testsatz unterteilen. Dazu nehmen wir einfach die ersten 1800 Moleküle als Trainingset und den Rest als Testset. Es ist wichtig, dass wir in diesem Beispiel keine Minibtaches verwenden. Der Grund dafür ist, dass die Featurematrizen der einzelnen Moleküle unterschiedlich groß sind bzw. eine unterschiedliche Anzahl von Zeilen haben. Das bedeutet, dass wir sie nicht einfach als 3D-Tensor speichern können, wie es zum Beispiel bei CNNs der Fall ist. 

Es gibt Möglichkeiten, dieses Problem zu lösen, aber sie sind für dieses Notebook nicht relevant. 

In [26]:
train_feat = feat[:1800]
train_DA = DA[:1800]
train_labels = labels[:1800]


test_feat = feat[1800:]
test_DA = DA[1800:]
test_labels = labels[1800:]

In [29]:
gnn = GraphNN()
loss_funktion= nn.BCEWithLogitsLoss()
optimizer=optim.Adam(gnn.parameters(), lr =0.01)

In [30]:
EPOCHS = 20

for i in range(EPOCHS):
    loss_list_train = []
    acc_list_train= []
    gnn.train()
    for k in range(len(train_feat)):
        optimizer.zero_grad()
    
        output=gnn(train_feat[k], train_DA[k]).flatten()

        loss=loss_funktion(output,train_labels[k])
        loss.backward()
        loss_list_train.append(loss.item())
        optimizer.step()

        acc_list_train.append(np.sum((torch.round(torch.sigmoid(output)) == train_labels[k]).detach().numpy()))
    loss_list_test = []
    acc_list_test= []
    gnn.eval()
    for k in range(len(test_feat)):
    
        output=gnn(test_feat[k], test_DA[k]).flatten()

        loss=loss_funktion(output,test_labels[k])
        loss_list_test.append(loss.item())
 

        acc_list_test.append(np.sum((torch.round(torch.sigmoid(output)) == test_labels[k]).detach().numpy()))
            
        
    print(i,"Train Loss: %.2f Train Accuracy: %.2f Test Loss: %.2f Test Accuracy: %.2f"
        % (np.mean(loss_list_train), np.mean(acc_list_train),np.mean(loss_list_test), np.mean(acc_list_test)))

0 Train Loss: 0.70 Train Accuracy: 0.52 Test Loss: 0.69 Test Accuracy: 0.50
1 Train Loss: 0.67 Train Accuracy: 0.59 Test Loss: 0.64 Test Accuracy: 0.63
2 Train Loss: 0.65 Train Accuracy: 0.63 Test Loss: 0.63 Test Accuracy: 0.66
3 Train Loss: 0.63 Train Accuracy: 0.66 Test Loss: 0.61 Test Accuracy: 0.67
4 Train Loss: 0.62 Train Accuracy: 0.67 Test Loss: 0.61 Test Accuracy: 0.67
5 Train Loss: 0.61 Train Accuracy: 0.68 Test Loss: 0.60 Test Accuracy: 0.68
6 Train Loss: 0.61 Train Accuracy: 0.68 Test Loss: 0.61 Test Accuracy: 0.67
7 Train Loss: 0.61 Train Accuracy: 0.69 Test Loss: 0.61 Test Accuracy: 0.68
8 Train Loss: 0.60 Train Accuracy: 0.69 Test Loss: 0.61 Test Accuracy: 0.67
9 Train Loss: 0.60 Train Accuracy: 0.69 Test Loss: 0.61 Test Accuracy: 0.67
10 Train Loss: 0.60 Train Accuracy: 0.69 Test Loss: 0.62 Test Accuracy: 0.67
11 Train Loss: 0.60 Train Accuracy: 0.69 Test Loss: 0.62 Test Accuracy: 0.67
12 Train Loss: 0.60 Train Accuracy: 0.69 Test Loss: 0.64 Test Accuracy: 0.67
13 Train 

Wie Sie sehen können, dauert das Training sehr lange und ist nicht sehr erfolgreich. Das hier vorgestellte Modell ist ein sehr, sehr einfaches Modell. In der Tat werden normalerweise komplexere Graphen Convolutions verwendet. Auch wird nicht nur der Atomtyp, sondern eine Vielzahl an Features als Input verwendet. 

Seit ein paar Jahren gibt es auch [PyTorch Geometric](https://pytorch-geometric.readthedocs.io/en/latest/) und die [Deep Graph Library](https://www.dgl.ai/). Beide stellen eine Erweiterung zu PyTorch dar. Sie enthalten wichtige Funktionalitäten, die für den Umgang mit Graphen relevant sind. Außerdem enthalten die Libraries die wichtigsten Graph Layers. Auf diese Weise müssen Sie die Layers nicht selbst programmieren.

## Übungsaufgabe:

Hier sehen Sie unser Graph Neural Network.
Das Problem ist, dass dieses Netzwerk keine Flexibilität bietet.
Die Weightmatrizen des Netzwerks werden immer die selber Größen haben.
Auch fehlt Dropout. Wir brauchen kein Batchnorm, da wir keine Minibatches benutzen.


```python
class GraphNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GraphConvolution(25, 100)
        self.conv2 = GraphConvolution(100, 100)
        self.lin = nn.Linear(100, 1)
        
    def aggregate(self, convoluted_graph): 
        return torch.mean(convoluted_graph, dim=0, keepdim=True)
        
    def forward(self, x, adj):
        x = self.conv1(x, adj)
        x = F.relu(x)
        x = self.conv2(x, adj)
        x = F.relu(x)
        x = self.aggregate(x)
        x = self.lin(x)
        return x
```

Können Sie das Netzwerk so umschreiben, dass wir flexibel den Input, als auch die größen der Graph Convolutions anpassen können?
Sie können testen, ob ihr Netzwerk noch funktioniert mit dem `beispiel_DA` und `beispie_feat`.

In [ ]:
beispiel_DA = test_DA[0]
beispiel_feat = test_feat[0]

In [ ]:
class GraphNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GraphConvolution(25, 100)
        self.conv2 = GraphConvolution(100, 100)
        self.lin = nn.Linear(100, 1)
        
    def aggregate(self, convoluted_graph): 
        return torch.mean(convoluted_graph, dim=0, keepdim=True)
        
    def forward(self, x, adj):
        x = self.conv1(x, adj)
        x = F.relu(x)
        x = self.conv2(x, adj)
        x = F.relu(x)
        x = self.aggregate(x)
        x = self.lin(x)
        return x